In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2

import keras
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD
from tensorflow.keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

import warnings
warnings.filterwarnings("ignore")




In [ ]:
def preprocess(img):
    width, height = img.shape[0], img.shape[1]
    img = image.array_to_img(img, scale=False)

    # Crop 48x48px
    desired_width, desired_height = 224, 224

    if width < desired_width:
        desired_width = width
    start_x = np.maximum(0, int((width-desired_width)/2))

    img = img.crop((start_x, np.maximum(0, height-desired_height), start_x+desired_width, height))
    img = img.resize((224, 224))

    img = image.img_to_array(img)
    return img / 255.

In [ ]:
def preprocess_data(train_dir, validate_dir):
    img_width, img_height = 224,224
    batch_size =16
    # data pre-processing for training
    train_datagen =  ImageDataGenerator(
        rescale=1./255,
        rotation_range = 10,
        zoom_range = [1.00, 1.00],
        horizontal_flip = True,
        fill_mode = 'nearest')
    

    # data pre-processing for validation
    validate_datagen =  ImageDataGenerator(rescale=1./255)

    # generate and store training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size)

    # generate and store validation data
    validate_generator = validate_datagen.flow_from_directory(
        validate_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size)
    
    return train_generator, validate_generator

In [ ]:
train_dir = "../input/chest-xray-pneumonia/chest_xray/train"
validation_dir = "../input/chest-xray-pneumonia/chest_xray/val"
test_dir = "../input/chest-xray-pneumonia/chest_xray/test"
# train_generator, validate_generator = preprocess_data(train_dir, validate_dir)# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(rotation_range=40,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical'
                                                        )

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=16,
                                                  class_mode='categorical'
                                                 )

In [ ]:
# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(rotation_range=40,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical'
                                                        )

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=16,
                                                  class_mode='categorical'
                                                 )

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.applications.resnet import ResNet101
from tensorflow.python.keras import Sequential

In [ ]:
#load pre trained Xception model
base_model=tf.keras.applications.resnet.ResNet101(include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
 #and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


# add a global spatial average pooling layer
#x = base_model.output
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
 #and a logistic layer -- let's say we have 200 classes
#predictions = Dense(2, activation='softmax')(x)
# this is the model we will train
#model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
learning = 0.0001

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=learning, beta_1=0.85, beta_2=0.999, amsgrad=False), loss='categorical_crossentropy',metrics=['accuracy'])
#Summary of Xception Model

import warnings
warnings.filterwarnings("ignore")

model.summary()



In [ ]:
%%time
NUM_EPOCHS = 30
batch_size=8
history = model.fit_generator(
    train_generator,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,
    callbacks=None)

In [ ]:
def test(model):
    
    predict_datagen =  ImageDataGenerator(rescale=1./255)
    test_dir = "../input/chest-xray-pneumonia/chest_xray/test"
    
    # generate and store validation data
    predict_generator = predict_datagen.flow_from_directory(
        test_dir,
        class_mode=None,
        target_size = (224, 224),
        shuffle=False)

    pred = model.predict_generator(predict_generator)
    predicted_class_indices=np.argmax(pred,axis=1)
    
    return predicted_class_indices

In [ ]:
%%time
predicted_class_indices = test(model)

In [ ]:
predict_datagen =  ImageDataGenerator(rescale=1./255,
                                     preprocessing_function=preprocess)
test_dir = "../input/chest-xray-pneumonia/chest_xray/test"

# generate and store validation data
predict_generator = predict_datagen.flow_from_directory(
    test_dir,
    class_mode=None,
    target_size = (224, 224),
    shuffle=False)

pred = model.predict_generator(predict_generator)
predicted_class_indices=np.argmax(pred,axis=1)



In [ ]:
import seaborn as sns
N = NUM_EPOCHS
plt.style.use("seaborn-white")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="test_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

plt.style.use("seaborn-white")
plt.figure()
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="test_acc")
plt.title("Training Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

LABELS = ["covid-19","normal"]

def show_confusion_matrix(validations, predictions):
    matrix = confusion_matrix(validations, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(matrix,
                cmap="coolwarm",
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

filenames = validation_generator.filenames
nb_samples = len(filenames)

Y_pred = model.predict_generator(validation_generator,(nb_samples//batch_size))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
show_confusion_matrix(validation_generator.classes, y_pred)

print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')

target_names = ["covid-19","normal"]
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))



# Plot linewidth.
lw = 2
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(validation_generator.classes, y_pred)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
       lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

plt.savefig('foo.png')
plt.savefig('foo.pdf')

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=predict_generator.filenames
results=pd.DataFrame({"ID":filenames,
                      "Label":predictions})
results